In [1]:
import pandas as pd
from contextlib import contextmanager
import time as timee
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

In [9]:
data_path='/home/m21_huangzijun/pythonprojs/sichuan/'
id_file=data_path+'data/0527/train/train_user.csv'
voc_file=data_path+'data/0527/train/train_voc.csv'
train_id=data_path+'data/train_user0.csv'
name=pd.read_csv(id_file)
voc=pd.read_csv(voc_file)
train_name=pd.read_csv(train_id)
voc=voc[['phone_no_m','opposite_no_m','call_dur','calltype_id','city_name','county_name','start_datetime']]

/tmp/ipykernel_3441819/3173109913.py:6: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  voc=pd.read_csv(voc_file)


In [10]:
#test
voc.dtypes

phone_no_m        object
opposite_no_m     object
call_dur           int64
calltype_id        int64
city_name         object
county_name       object
start_datetime    object
dtype: object

In [11]:
#数据类型转换，数据清洗
voc.dropna(inplace=True)
voc['location']=voc['city_name']+' '+voc['county_name']
voc['start_datetime']=pd.to_datetime(voc['start_datetime'])

n_target_no=voc['opposite_no_m'].nunique()
n_location=voc['location'].nunique()



/tmp/ipykernel_3441819/3094468760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voc.dropna(inplace=True)


In [14]:
#test
n_location

197

In [15]:
# 编码
encoder=LabelEncoder().fit(voc['opposite_no_m'])
voc['opposite_no_m']=encoder.transform(voc['opposite_no_m'])

encoder=LabelEncoder().fit(voc['location'])
voc['location']=encoder.transform(voc['location'])

voc=voc.merge(name[['phone_no_m','label']],how='left',on='phone_no_m')



TypeError: string indices must be integers

In [19]:
#分开训练、测试voc
train_voc=voc.loc[voc['phone_no_m'].isin(train_name['phone_no_m'])]
test_voc=voc.loc[~(voc['phone_no_m'].isin(train_name['phone_no_m']))]

In [23]:
train_vocs=[g for _,g in train_voc.groupby('phone_no_m')]
test_vocs=[g for _,g in test_voc.groupby('phone_no_m')]


In [38]:
train_x=[]
train_y=[]
test_x=[]
test_y=[]

数据顺序
对端号码，地点，通话类型，通话时长，通话时间

In [40]:
for p in train_vocs:
    p.sort_values(by='start_datetime',inplace=True)
    #处理时间
    first_time=p['start_datetime'].iloc[0]
    p['time']=(p['start_datetime']-first_time).dt.total_seconds()#转化为s
    max_min_scalar=lambda x:(x-np.min(x))/(np.max(x)-np.min(x))
    p['time']=p[['time']].apply(max_min_scalar) 
    #处理时长
    p['call_dur']=p[['call_dur']].apply(max_min_scalar)
    #输出信息
    train_x.append(p[['opposite_no_m','location','calltype_id','call_dur','time']].to_numpy())
    train_y.append(p['label'].iloc[0])

In [41]:
for p in test_vocs:
    p.sort_values(by='start_datetime',inplace=True)
    #处理时间
    first_time=p['start_datetime'].iloc[0]
    p['time']=(p['start_datetime']-first_time).dt.total_seconds()#转化为s
    max_min_scalar=lambda x:(x-np.min(x))/(np.max(x)-np.min(x))
    p['time']=p[['time']].apply(max_min_scalar) 
    #处理时长
    p['call_dur']=p[['call_dur']].apply(max_min_scalar)
    #输出信息
    test_x.append(p[['opposite_no_m','location','calltype_id','call_dur','time']].to_numpy())
    test_y.append(p['label'].iloc[0])

In [52]:
n_target_no

1062387

In [53]:
n_location

197

In [51]:
#保存
np.savez(data_path+'data_after/mymodel2/8month.npz',train_x=train_x,train_y=train_y,test_x=test_x,test_y=test_y)

/home/m21_huangzijun/anaconda3/envs/fraud/lib/python3.9/site-packages/numpy/lib/npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [35]:
#test
df1=pd.DataFrame({'time':[1,2,3,4],'time2':[2,3,4,5]})
df1_records=df1.to_numpy()


In [48]:
#test
for x in test_x[:15]:
    print(len(x[0]))

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [43]:
#test
len(test_x)

1072